In [16]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import re

In [17]:
page = requests.get('https://www.newtral.es/zona-verificacion/fakes/')

In [18]:
#print ("pagina", page)
print ("pagina status code", page.status_code)
#print ("contenido de la pagina", page.content)

pagina status code 200


In [19]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup)
#print(soup.prettify())

In [20]:
def get_urls(link, lista_enlaces):
    
    page = requests.get(link)
    
    soup2 = BeautifulSoup(page.content, 'html.parser')

    todo2 = soup2.find('div', class_='post-cards-list')
    
    print(todo2)
    
    
    '''
    urls = todo2.find_all('a')

    enlaces = []
    for url in urls:
        #print(url.attrs['href'])
        enlaces.append(url.attrs['href'])
    for e in enlaces[1::2]:
        lista_enlaces.append(e)
    lista_enlaces
    '''

In [58]:
data_params = {    
    'per_page': '12',
    'offset': '12',
    'categories': '59',
    'exclude': ['111489','111567','110601','111313','111200','110789','111053','110921','110942','110963','110881','110688','110657','13775'],
    '_locale': 'user'
}   

page = requests.get('https://www.newtral.es/zona-verificacion/fakes/', params=data_params)

soup2 = BeautifulSoup(page.content, 'html.parser')

todo2 = soup2.find('div', class_='o-gallery js-gallery-content')

urls = todo2.find_all('a')

enlaces = []

for url in urls:
    #print(url.attrs['href'])
    enlaces.append(url.attrs['href'])
lista_enlaces = []   
for e in enlaces:
    lista_enlaces.append(e)
lista_enlaces


['https://www.newtral.es/documentos-que-parecen-oficiales-pero-estan-manipulados-ii/20200523/',
 'https://www.newtral.es/princesa-leonor-movil-60000-euros/20200522/',
 'https://www.newtral.es/el-uso-de-mascarillas-no-produce-hipoxia/20200521/',
 'https://www.newtral.es/bulo-video-ministro-frances-entrevista-bano/20200521/',
 'https://www.newtral.es/bulo-video-nacimiento-bebe-bolsa-amniotica-coronavirus/20200521/',
 'https://www.newtral.es/covid-19-epidemia-mundial-de-bulos-xx/20200521/',
 'https://www.newtral.es/bulo-gel-hidroalcoholico-explota/20200520/',
 'https://www.newtral.es/no-el-final-del-periodo-de-sesiones-del-congreso-no-haria-que-el-estado-de-alarma-dure-hasta-octubre/20200520/',
 'https://www.newtral.es/bulo-cupones-carrefour-phishing/20200520/',
 'https://www.newtral.es/el-gobierno-no-ha-acordado-a-20-de-mayo-una-subida-del-irpf-y-la-supresion-de-extras-a-funcionarios-y-pensionistas/20200520/',
 'https://www.newtral.es/bulo-vales-leroy-merlin-mil-euros-phishing/20200520/'

In [57]:
page = requests.get('https://www.newtral.es/wp-json/wp/v2/posts?per_page=12&offset=108&categories=59&exclude=111489%2C111567%2C110601%2C111313%2C111200%2C110789%2C111053%2C110921%2C110942%2C110963%2C110881%2C110688%2C110657%2C13775&_locale=user')
page.text

'[{"id":91642,"date":"2020-03-30T19:44:04","date_gmt":"2020-03-30T17:44:04","guid":{"rendered":"https:\\/\\/www.newtral.es\\/?p=91642"},"modified":"2020-05-21T13:10:24","modified_gmt":"2020-05-21T11:10:24","slug":"el-mensaje-falso-del-anciano-sin-documentacion-que-no-recuerda-nada-y-que-esta-en-el-hospital-la-paz","status":"publish","type":"post","link":"https:\\/\\/www.newtral.es\\/el-mensaje-falso-del-anciano-sin-documentacion-que-no-recuerda-nada-y-que-esta-en-el-hospital-la-paz\\/20200330\\/","title":{"rendered":"El mensaje falso del anciano \\u00absin documentaci\\u00f3n, que \\u00abno recuerda nada\\u00bb y que \\u00abest\\u00e1 en el Hospital La Paz\\u00bb"},"content":{"rendered":"\\n<p>Nos hab\\u00e9is preguntado a trav\\u00e9s de nuestro servicio de verificaci\\u00f3n de WhatsApp (+34 682 58 96 64) por el mensaje que acompa\\u00f1a la foto de un hombre mayor. En \\u00e9l se asegura que el anciano \\u201cno tiene documentaci\\u00f3n\\u201d, <strong>\\u201cno sabe qui\\u00e9n es